In [ ]:
import numpy as np
import matplotlib.pyplot as lt
import cv2
import scipy.linalg as lg
from google.colab.patches import cv2_imshow 

In [ ]:
def msc(y,m,x,c):
  result = y-(m*x)-c
  return result

In [ ]:
def horizondetect(himg,xsize,ysize):
  slope = np.linspace(-1,1,100)
  inter = np.linspace(1,ysize,100)
  maximum = []
  r_max = 0
  # print("printing")
  # print(himg)
  for m in range(len(slope)):
    for c in range(len(inter)):
      sky = []
      ground = []
      for i in range(xsize):
        for j in range(ysize):
          u = msc(j,slope[m],i,inter[c])
          v = (-1*inter[c])
          if(u*v>0):
            sky.append(himg[j][i])
          else:
            ground.append(himg[j][i])
      
      sky = np.transpose(sky)
      ground = np.transpose(ground)

      try:
        co_sky = np.cov(sky)
        co_ground = np.cov(ground)
        co_sky_det = lg.det(co_sky)
        co_ground_det = lg.det(co_ground)
        co_sky_eig = lg.eig(co_sky)
        co_ground_eig = lg.eig(co_ground)

        r = 1/(co_sky_det + co_ground_det + (co_sky_eig[0][0]+co_sky_eig[0][1]+co_sky_eig[0][2])**2 + (co_ground_eig[0][0]+co_ground_eig[0][1]+co_ground_eig[0][2])**2)
        if (r > r_max):
          r_max = r
          maximum = [slope[m],inter[c]]
      except Exception:
        pass
  return maximum

In [ ]:
def drawline(himg , horizon):
  xsize = himg.shape[1]
  print("xsize",xsize)
  m = horizon[0]
  b = horizon[1]
  y2 = int(m*(xsize-1)+b)
  cv2.line(himg ,(0,int(b)),(xsize-1 , y2),(0,0,255) , 1)
  cv2_imshow(himg)

In [ ]:
path ="/content/image.jpg"
himage_org = cv2.imread(path)
cv2_imshow(himage_org)
himage = cv2.resize(himage_org , (0,0) , fx = 1/10 , fy = 1/10)
ysize = himage.shape[0]
xsize = himage.shape[1]
print("values of sizes")
print (ysize , xsize)
horizon = []
horizon = horizondetect(himage , xsize, ysize)
horizon[1] = horizon[1]*10
print(horizon)
drawline(himage_org , horizon)